In [1]:
# Declare Dependencies 
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

## Mac Users Run This Code

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

## Windows Users Run This Code

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Mars Data Scraping from NASA News Page

In [4]:
# Set URL to NASA Mars News website
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [5]:
# Create HTML Object and parse HTML with Beautiful Soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Scrape data from the title and p tags
text_tag = soup.find('div', class_='list_text')
title = text_tag.find('div', class_='content_title')
article_title = title.find('a').text
article_p = soup.find('div', class_='article_teaser_body').text

# Print scraped data for verification
print(article_title)
print(article_p)

NASA's Perseverance Rover Mission Getting in Shape for Launch
Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. 


## Mars Image Scraping from JPL Page

In [6]:
# Set URL to JPL website
image_url_featured = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
image_url_prefix = 'jpl.nasa.gov'
browser.visit(image_url_featured)

In [7]:
# Create HTML Object and parse HTML with Beautiful Soup 
html_weather = browser.html
soup = BeautifulSoup(html_weather, 'html.parser')

In [8]:
image_carousel = soup.find('article', class_='carousel_item')
footer = image_carousel.find('footer')
link_url = footer.find('a')["data-fancybox-href"]
featured_image_url = image_url_prefix + link_url
featured_image_url

'jpl.nasa.gov/spaceimages/images/mediumsize/PIA16227_ip.jpg'

## Mars Weather Scraping from Twitter

In [9]:
# Set URL to Mars Weather Twitter
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [10]:
# Create HTML Object and parse HTML with Beautiful Soup 
html_weather = browser.html
soup = BeautifulSoup(html_weather, 'html.parser')

In [11]:
# Find all tweets through BS
latest_tweet = soup.find('article', role="article")
mars_weather = latest_tweet.find_all('span')[4].text

# Print scraped data for verification
print(mars_weather)

InSight sol 522 (2020-05-16) low -93.2ºC (-135.7ºF) high -1.7ºC (29.0ºF)
winds from the SW at 5.1 m/s (11.3 mph) gusting to 18.8 m/s (42.1 mph)
pressure at 7.00 hPa


## Mars Data Scraping from Mars Facts Page

In [12]:
# Set URL to Mars Facts website
facts_url = 'http://space-facts.com/mars/'

In [13]:
# Use Panda's `read_html` to parse the url
mars_facts = pd.read_html(facts_url)

In [14]:
mars_facts[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
# Find the mars facts DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = mars_facts[0]

In [16]:
# Assign the columns `['Description', 'Value']`
mars_df.columns = ['Description','Value']

In [17]:
# Reassign index to the `Description` column
mars_df.set_index('Description', inplace=True)

In [18]:
# Display mars_df
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
# Save html code
mars_df.to_html('mars_data.html')
data = mars_df.to_dict(orient='records')

## Mars Hemisphere Image Scraping from usgs.gov

In [20]:
# Set URL to usgs.gov website
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [21]:
# Create HTML Object and parse HTML with Beautiful Soup
html_weather = browser.html
soup = BeautifulSoup(html_weather, 'html.parser')

In [22]:
# Declare list variables for storing image URLs dictionary
hemisphere_image_urls = []
hemisphere_url_prefix = 'https://astrogeology.usgs.gov'

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemisphere_url_prefix + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemisphere_url_prefix + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]